# Code Utilized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import statsmodels.api as sm

In [ ]:
conn = sqlite3.connect('TexasStops.db')
curs = conn.cursor()

tables = curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

print('Tables in Database:')
for table in tables:
    print(table[0])
    
#Lubbock has no race data

In [ ]:
query = """SELECT 
            subject_race, 
            CAST(SUBSTR(date, 1, 4) as INT) AS year,
            COUNT(subject_race),
            SUM(CASE WHEN search_conducted = 'TRUE' THEN 1 ELSE 0 END) as num_searched,
            SUM(CASE WHEN warning_issued = 'TRUE' THEN 1 ELSE 0 END) as num_warned,
            SUM(CASE WHEN citation_issued = 'TRUE' THEN 1 ELSE 0 END) as num_cited
            FROM Statewide 
            WHERE year = 2010 AND type = 'vehicular'
            GROUP BY subject_race, year

        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['Race', 'Year', 'num_stopped','num_searched','num_warned',
                                       'num_cited'])

In [ ]:
#this array is percentage of each race according to 2010 census
array = np.array([3.9, 11.8, 37.6, 11.8, 10.5, 45.3]) 
total_pop  = 25145561 # total texas population as of 2010 census
total_nums = total_pop * array

df['population'] = total_nums
df['Stop rate'] = df['num_stopped'] / df['population']
df['Search rate'] = df['num_searched'] / df['population']
df['Warning rate'] = df['num_warned'] / df['population']
df['Citation rate'] = df['num_cited'] / df['population']

df

In [ ]:
query = """SELECT 
            CASE WHEN subject_race = 'white' then 'White' else 'Minority' end as is_white, 
            SUM(CASE WHEN search_conducted = 'TRUE' THEN 1 ELSE 0 END) as num_searched, 
            SUM(CASE WHEN contraband_found = 'TRUE' THEN 1 ELSE 0 END) as num_success,
            (SUM(CASE WHEN contraband_found = 'TRUE' THEN 1 ELSE 0 END)*1.0 / SUM(CASE WHEN search_conducted = 'TRUE' THEN 1 ELSE 0 END)) as hit_rate
            FROM Statewide 
        GROUP BY is_white
        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['Race', 'Number Searched', 'Number Contraband Found', 'Hit Rate'])
df

In [ ]:
query = """SELECT 
            CAST(SUBSTR(date, 1, 4) as INT) AS year,
            CASE WHEN subject_race = 'white' then 'White' else 'Minority' end as is_white, 
            SUM(CASE WHEN search_conducted = 'TRUE' THEN 1 ELSE 0 END) as num_searched, 
            SUM(CASE WHEN contraband_found = 'TRUE' THEN 1 ELSE 0 END) as num_success,
            (SUM(CASE WHEN contraband_found = 'TRUE' THEN 1 ELSE 0 END)*1.0 / SUM(CASE WHEN search_conducted = 'TRUE' THEN 1 ELSE 0 END)) as hit_rate
        FROM Statewide 
        GROUP BY year, is_white
        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['Year','Race', 'Number Searched', 'Number Contraband Found', 'Hit Rate'])

In [ ]:
plt.clf()

for race in df['Race'].unique():
    race_df = df.loc[df['Race'] == race]
    plt.scatter(race_df['Year'],race_df['Hit Rate'], label = race)
    
plt.legend()
plt.title('Hit Rate changes in Texas by Year')
plt.xlabel('Year')
plt.ylabel('Hit Rate')
plt.show()

In [ ]:
query = """SELECT 
            county_name,
            COUNT(*) as num_stops,
            SUM(CASE WHEN (contraband_found = 'TRUE' AND subject_race = 'white') 
                    THEN 1 ELSE 0 END) * 1.0 / SUM(CASE WHEN (search_conducted = 'TRUE' 
                        AND subject_race = 'white') THEN 1 ELSE 0 END) as hit_rate_white,
    
            SUM(CASE WHEN (contraband_found = 'TRUE' AND subject_race != 'white') 
                THEN 1 ELSE 0 END) * 1.0 / SUM(CASE WHEN (search_conducted = 'TRUE' 
                    AND subject_race != 'white') THEN 1 ELSE 0 END) as hit_rate_minority

            FROM Statewide 
            GROUP BY county_name
            HAVING hit_rate_white IS NOT NULL and hit_rate_minority IS NOT NULL 
            and hit_rate_white > 0 and hit_rate_minority > 0 
        and hit_rate_white < 1 and hit_rate_minority < 1
        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['County','Number of Stops','White hit rate', 'Minority hit rate'])
df.head()

In [ ]:
plt.clf()
plt.scatter(df['White hit rate'], df['Minority hit rate'], s = (df['Number of Stops'] / 3000))
x = np.linspace(0,0.7)
plt.plot(x,x, color = 'red')
plt.xlabel('White hit rate')
plt.ylabel('Minority hit rate')
plt.title('Minority Hit rates vs White hit rates by Texas County')
plt.show

In [ ]:
print(df.shape[0])
bias_counties = df.loc[df['White hit rate'] > df['Minority hit rate']]
print(bias_counties.shape[0])

print(bias_counties.shape[0]/ df.shape[0])

In [ ]:
query = """SELECT 
            officer_id_hash,
            COUNT(*) as num_stops,
            SUM(CASE WHEN (contraband_found = 'TRUE' AND subject_race = 'white') 
                    THEN 1 ELSE 0 END) * 1.0 / SUM(CASE WHEN (search_conducted = 'TRUE' 
                        AND subject_race = 'white') THEN 1 ELSE 0 END) as hit_rate_white,
    
            SUM(CASE WHEN (contraband_found = 'TRUE' AND subject_race != 'white') 
                THEN 1 ELSE 0 END) * 1.0 / SUM(CASE WHEN (search_conducted = 'TRUE' 
                    AND subject_race != 'white') THEN 1 ELSE 0 END) as hit_rate_minority

            FROM Statewide 
            GROUP BY officer_id_hash
            HAVING hit_rate_white IS NOT NULL and hit_rate_minority IS NOT NULL 
            and hit_rate_white > 0 and hit_rate_minority > 0 
        and hit_rate_white < 1 and hit_rate_minority < 1
        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['officer','Number of Stops','White hit rate', 'Minority hit rate'])
df.head()

In [ ]:
plt.clf()
plt.scatter(df['White hit rate'], df['Minority hit rate'], s = (df['Number of Stops'] / 3000))
x = np.linspace(0,0.9)
plt.plot(x,x, color = 'red')
plt.xlabel('White hit rate')
plt.ylabel('Minority hit rate')
plt.title('Minority Hit rates vs White hit rates by Texas state trooper')
plt.show

In [ ]:
df['hit_rate_difference'] = df['White hit rate'] - df['Minority hit rate']
std_dev = round(df['hit_rate_difference'].std(), 3)
mean = round(df['hit_rate_difference'].mean(), 3)
plt.hist(df['hit_rate_difference'], bins = 40)
plt.title('Histogram of Hit rate differences, ' +'Mean = ' + str(mean) +  ' Std Dev = ' + str(std_dev))
plt.xlabel('Hit rate difference')
plt.ylabel('Police officer frequency')

In [ ]:
sm.qqplot(df['hit_rate_difference'], line = 's') 
plt.title('qq plot of Hit rate difference distribution')
plt.plot()

In [ ]:
query = """SELECT 
            CASE WHEN contraband_found = 'TRUE' then 1 else 0 end as contraband_found,
            CASE WHEN subject_race = 'white' then 1 else 0 end as is_white, 
            CASE WHEN subject_sex = 'male' then 1 else 0 end as is_male
            FROM Statewide 
            WHERE search_conducted = 'TRUE'
        """
query_res = curs.execute(query).fetchall()

df = pd.DataFrame(query_res, columns = ['Contraband_found','is white','is male'])
df.head()

In [ ]:
#fit logistic regression model 
y = df['Contraband_found']
X = df[['is white','is male']]


model = sm.Logit(y,sm.add_constant(X)).fit()
model.summary()